In [1]:
import ast
import os

import nbformat

from config import ROOT


def find_function_defs(py_paths):
    """Find all function definitions in provided .py files. Return dict: {func_name: [file1, file2, ...]}"""
    defs = {}
    for path in py_paths:
        with open(path, encoding="utf-8") as f:
            try:
                tree = ast.parse(f.read())
                for node in ast.walk(tree):
                    if isinstance(node, ast.FunctionDef):
                        defs.setdefault(node.name, []).append(path)
            except Exception as e:
                print(f"Error parsing {path}: {e}")
    return defs


def find_function_calls_in_py(py_paths):
    """Find all function call names in provided .py files."""
    calls = set()

    class CallVisitor(ast.NodeVisitor):
        def visit_Call(self, node):
            # foo() or obj.foo()
            if isinstance(node.func, ast.Name):
                calls.add(node.func.id)
            elif isinstance(node.func, ast.Attribute):
                calls.add(node.func.attr)
            self.generic_visit(node)

    for path in py_paths:
        with open(path, encoding="utf-8") as f:
            try:
                tree = ast.parse(f.read())
                CallVisitor().visit(tree)
            except Exception as e:
                print(f"Error parsing {path}: {e}")
    return calls


def find_function_calls_in_ipynb(ipynb_paths):
    """Find all function call names in provided .ipynb files."""
    calls = set()

    class CallVisitor(ast.NodeVisitor):
        def visit_Call(self, node):
            if isinstance(node.func, ast.Name):
                calls.add(node.func.id)
            elif isinstance(node.func, ast.Attribute):
                calls.add(node.func.attr)
            self.generic_visit(node)

    for path in ipynb_paths:
        try:
            with open(path, encoding="utf-8") as f:
                nb = nbformat.read(f, as_version=4)
                for cell in nb.cells:
                    if cell.cell_type == "code":
                        try:
                            tree = ast.parse(cell.source)
                            CallVisitor().visit(tree)
                        except Exception:
                            continue
        except Exception as e:
            print(f"Error reading {path}: {e}")
    return calls


def collect_paths(root, ext):
    """Recursively collect all file paths with a given extension."""
    paths = []
    for dirpath, _, files in os.walk(root):
        for file in files:
            if file.endswith(ext):
                paths.append(os.path.join(dirpath, file))
    return paths


if __name__ == "__main__":
    # Adjust these roots as needed
    py_paths = collect_paths(ROOT / "code/src", ".py")
    ipynb_paths = collect_paths(ROOT / "code/", ".ipynb")  # Looks for notebooks in current dir and subdirs

    func_defs = find_function_defs(py_paths)
    calls_py = find_function_calls_in_py(py_paths)
    calls_ipynb = find_function_calls_in_ipynb(ipynb_paths)

    all_calls = calls_py | calls_ipynb
    unused = [fn for fn in func_defs if fn not in all_calls]

    print("\nUnused functions (defined in src/, never called in .py or .ipynb):")
    for fn in sorted(unused):
        locations = func_defs[fn]
        for loc in locations:
            print(f"  {fn:25} defined in {loc}")



Unused functions (defined in src/, never called in .py or .ipynb):
  compute_row_score         defined in C:\Users\tm7202\Workspace\replication-sari-forecasting\code\src\scoring_functions.py
  extract_info              defined in C:\Users\tm7202\Workspace\replication-sari-forecasting\code\src\load_data.py
  get_preceding_thursday    defined in C:\Users\tm7202\Workspace\replication-sari-forecasting\code\src\realtime_utils.py
  set_last_n_values_to_nan  defined in C:\Users\tm7202\Workspace\replication-sari-forecasting\code\src\realtime_utils.py
